In [119]:
import torch 
from torch import nn
import torch.nn.functional as F

In [2]:
device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")

In [45]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convblock1 = nn.Sequential(
                nn.Conv2d(3,8,2,stride=2),
                nn.ReLU(False)
        )
        
        
        self.convblock2 = nn.Sequential(
                nn.Conv2d(8,16,2,stride=2),
#                 nn.MaxPool2d(2,2),
                nn.ReLU(True)
        )
        
        
        self.convblock3 = nn.Sequential(
                nn.Conv2d(16,32,2,stride=2),
                nn.MaxPool2d(2,2),
                nn.ReLU(True)
        )
        
    def forward(self,x):
        x = self.convblock1(x)
        print("v1",x.shape)
        x = self.convblock2(x)
        print("v2",x.shape)
        x = self.convblock3(x)
        print("v3",x.shape)
        flatten = x.view(2*2*32,-1)
        return x,flatten

model = Network().to(device)

In [46]:
model.convblock1[0].weight.shape

torch.Size([8, 3, 2, 2])

In [47]:
x = torch.randn((1,3,32,32),dtype=torch.float).to(device)

In [48]:
x,flatten = model(x)

v1 torch.Size([1, 8, 16, 16])
v2 torch.Size([1, 16, 8, 8])
v3 torch.Size([1, 32, 2, 2])


In [115]:
class ConvBlock(nn.Sequential):
    def __init__(self, in_filters, out_filters, bn=True, kernel_size=3, stride=2):
        layers = [nn.Conv2d(in_filters, out_filters, kernel_size=kernel_size, stride=stride)]
        if bn:
            layers.append(nn.BatchNorm2d(out_filters, 0.8))
        layers.append(nn.LeakyReLU(0.2, inplace=True))

        super().__init__(*layers)

In [116]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = ConvBlock(3,8,kernel_size=2,stride=2)
        self.conv2 = ConvBlock(8,16,kernel_size=2,stride=2)
        self.conv3 = ConvBlock(16,32,kernel_size=2,stride=2)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return x
        

In [117]:
model = Network().to(device)

In [114]:
x = torch.randn((1,3,32,32),dtype=torch.float).to(device)

In [110]:
model(x).shape

torch.Size([1, 32, 4, 4])

In [120]:
x = torch.randn((1,3,32,32),dtype=torch.float).to(device)

In [123]:
y = F.pad(x,(1,1,2,2))
# Examples::

#     >>> t4d = torch.empty(3, 3, 4, 2)
#     >>> p1d = (1, 1) # pad last dim by 1 on each side
#     >>> out = F.pad(t4d, p1d, "constant", 0)  # effectively zero padding
#     >>> print(out.data.size())
#     torch.Size([3, 3, 4, 4])
#     >>> p2d = (1, 1, 2, 2) # pad last dim by (1, 1) and 2nd to last by (2, 2)
#     >>> out = F.pad(t4d, p2d, "constant", 0)

In [125]:
y.shape

torch.Size([1, 3, 36, 34])

In [127]:
x.shape

torch.Size([1, 3, 32, 32])